# Baseline 모델 

### 0. 패키지 설치 / 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
# random seed 고정 
np.random.seed(2018)

In [3]:
# 현재 디렉토리 확인 
import os
print (os.getcwd())

C:\Users\jiwon


In [5]:
# 데이터를 불러온다. 
trn = pd.read_csv('C:/Users/jiwon/data/train_ver2.csv') # 다운 받은 파일이 해당 경로에 있는지 확인

C:\Users\jiwon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
tst = pd.read_csv('C:/Users/jiwon/data/test_ver2.csv') 

C:\Users\jiwon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1. 데이터 전처리 

In [7]:
# 제품 변수를 별도로 저장해 놓는다.
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체한다.
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터를 제거한다.
no_product = trn[prods].sum(axis=1) == 0  # axis=1은 y축 합계 
trn = trn[~no_product]

In [8]:
# 훈련 데이터와 테스트 데이터를 통합한다. 테스트 데이터에 없는 제품 변수는 0으로 채운다.
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

In [9]:
# 학습에 사용할 변수를 담는 list이다.
features = []

In [10]:
# 범주형 변수를 .factorize() 함수를 통해 label encoding한다.
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)  # na_sentinel은 missing value가 -99로 표현되도록 만든다. 이때 -99는 unique한 값으로 여겨지지 않는다. 
features += categorical_cols

In [26]:
## 참고: na_sentinel
codes, uniques = pd.factorize(['b', None, 'a', 'c', 'b'])
print(codes)
print(uniques)

[ 0 -1  1  2  0]
['b' 'a' 'c']


In [11]:
# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환한다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)  # 고객 구분이 1,2,3,4,P등급으로 되어있는데, 여기서 P등급을 5로 바꾸어 줌 
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [12]:
# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente'] 

### 2. 피쳐 엔지니어링 

피쳐 엔지니어링 단계에서는 모델 학습에 사용한 파생 변수를 생성한다.
ex) 날짜/시간 정보 → 주중/주말 여부, 공휴일 여부, 아침/낮/밤, 봄/여름/가을/겨울 등 

Baseline 모델에서는 전체 24개의 고객 변수와, 4개의 날짜 변수 기반 파생 변수, 그리고 24개의 lag-1 변수를 사용한다. 

#### 날짜 변수 기반 파생 변수

In [13]:
# 2개의 날짜 변수에서 연도와 월 정보를 추출한다.
# fecha_alta_month(고객이 첫 계약을 맺은 날짜), ult_fec_cli_1t_month(고객이 마지막으로 1등급이었던 날짜)

# fetcha_alta 변수형이 float(실수)이면 0을 반환하고, 아닐 경우 '-'를 기준으로 나누어 2번째 값을 정수형으로 표현한 뒤, fetcha_alta_month로 저장 
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8) 
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

*참고: 데이터 효율을 위해 month와 year을 각각 int8과 int16으로 표현한 것으로 보인다. int8과 int16는 각각 2^8개, 2^16개의 정수 표현이 가능하다. 즉, 각각 -128부터 127까지, -32768부터 32767까지의 정수를 표현할 수 있다. 범위가 1부터 12까지인 월을 표현할 때는 int8로 충분히 표현이 가능하지만(int16을 사용하여 메모리를 불필요하게 많이 쓰지 않아도 됨), 연도는 네 자리 수이기 때문에 int16을 사용하여 정수로 변환한 것으로 보인다. 

이외에도 날짜 변수를 활용하여 다양한 파생 변수를 생성할 수 있다. 예를 들어 두 개의 날짜 변수 간의 차이값을 파생 변수를 생성한다거나, 졸업식이나 방학 등의 특별한 날짜까지의 거리를 수치형 변수로 생성할 수 있다. 

#### 결측값 처리

In [14]:
# 그 외 변수의 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

사이킷런에서 제공하는 머신러닝 모델은 결측값을 입력값으로 받지 않고 실행 에러가 발생한다. 하지만 xgboost 모델에서는 결측값도 하나의 정보로 인식하고 모델 학습에 활용한다. 그러나 저자는 결측값을 -99로 설정함.

#### lag-1 파생 변수

시계열 데이터라는 데이터 특성을 살려 고객의 과거 데이터를 기반으로 다양한 파생 변수를 생성할 수 있다. 예를 들어, 고객의 나이가 최근 3개월 동안 변동이 있었는지(즉, 3개월 안에 생일을 맞이했는지)를 이진 변수로 생성하거나, 한 달 전에 구매한 제품에 대한 정보를 변수로 사용할 수 있고, 최근 6개월 간 평균 월급을 계산할 수도 있다. 

산탄데르 제품 추천 경진대회에서는 N개월 전에 금융 제품을 보유하고 있었는지 여부를 나타내는 lag 변수가 좋은 파생 변수로 작용했다. Baseline 모델에서는 1개월 전 정보만을 가져다 사용하는 lag-1 변수를 사용한다. 

In [15]:
# 날짜를 숫자로 변환하는 함수를 생성한다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")]  # '-' 기준으로 날짜를 나눈 뒤 리스트 형태로 각각 Y, M, D에 저장 
    int_date = (int(Y) - 2015) * 12 + int(M)  # 2015년 기준 개월 수를 계산 
    return int_date

In [16]:
# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [17]:
# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. ncodpers과 int_date를 제외한 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1  # 만일 lag-2, lag-5를 만들고 싶다면, 여기에 있는 숫자를 2나 5로 바꾸면 된다. 

In [18]:
df.head(2)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,fecha_alta_month,fecha_alta_year,ult_fec_cli_1t_month,ult_fec_cli_1t_year,int_date
0,2015-01-28,1375586,0,0,0,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,1,2015,0,0,1
1,2015-01-28,1050611,0,0,1,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,8,2012,0,0,1


In [19]:
df_lag.head(2)

,fecha_dato_prev,ncodpers,ind_empleado_prev,pais_residencia_prev,sexo_prev,age_prev,fecha_alta_prev,ind_nuevo_prev,antiguedad_prev,indrel_prev,...,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,fecha_alta_year_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev,int_date
0,2015-01-28,1375586,0,0,0,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,1,2015,0,0,2
1,2015-01-28,1050611,0,0,1,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,8,2012,0,0,2


예를 들어 날짜가 2015년 1월 28일이었다면 fecha_dato 변수에 들어간 값은 날짜형인 2015-01-28이었을 것이고, 이를 date_to_int 함수로 변환한 값인 1이 int_date에 저장된다. 그리고 _prev 변수에는 여기에 1인 더한 값인 2가 들어간다. 

fecha_dato였던 변수명이 fecha_dato_prev으로 바뀌었으며, 다른 변수들도 마찬가지이다. 

In [20]:
# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# 이 명령어 실행시키는 데 시간이 오래 걸리니, 유의하세요. (저는 30분 정도 걸렸습니다)

두 데이터셋을 합쳐 변수가 104개인 df_trn 데이터셋이 만들어진다.

In [21]:
# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
del df, df_lag

In [23]:
# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)

In [24]:
df_trn.fillna(-99, inplace=True)  

In [25]:
# lag-1 변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

###
### Baseline 모델 이후, 다양한 피쳐 엔지니어링을 여기에 추가한다.
###

### 3. 모델 학습 및 예측

#### 교차검증
교차검증은 경진대회에서 좋은 성적을 거두기 위해 매우 중요. 올바른 교차 검증 과정을 통해 제한 없이 다양한 아이디어를 실험하고, 성능 개선 여부를 확인하는 것이 매우 중요.

훈련 데이터: 2015-01-28 ~ 2016-05-28 (1년 6개월치) 테스트 데이터: 2018-06-28 (미래 데이터) --> 내부 교차검증 과정에서도 최신 데이터 (2016-05-28)를 검증 데이터로 분리, 나머지 데이터를 훈련 데이터로 사용

In [ ]:
## 모델 학습
# 학습을 위하여 데이터를 훈련, 테스트용으로 분리한다.
# 학습에는 2016-01-28 ~ 2016-04-28 데이터만 사용하고, 검증에는 2016-05-28 데이터를 사용한다.
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)] #isin 구문을 통해 use_dates 리스트 속 날짜들을 포함하고 있는 row만을 뽑아내 trn에 저장.
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

In [ ]:
# 훈련 데이터에서 신규 구매 건수만 추출한다.
X = []
Y = []
for i, prod in enumerate(prods): #enumerate: 순서가 있는 자료형(리스트, 튜플, 문자열)을 입력받아 인덱스값에 포함시키는 역할
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]  #해당 시점 제품 보유 O & 한달 전 제품 보유 X 인 경우 즉 신규 제품 구매한 row만 뽑아낸다
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i #신규 구매에 대한 label 값
    X.append(prX)
    Y.append(prY)  
XY = pd.concat(X) #Dataframe 형태로 만들어주는듯(?)
Y = np.hstack(Y)
XY['y'] = Y #XY는 신규 구매가 있었던 시점의 데이터

In [ ]:
# 훈련, 검증 데이터로 분리한다. 
vld_date = '2016-05-28'

XY_trn = XY[XY['fecha_dato'] != vld_date] #훈련 데이터
XY_vld = XY[XY['fecha_dato'] == vld_date] #검증 데이터

### 사용 모델: XGBoost
eXtreme Gradient Boosting: 

- Gradient Boosting 알고리즘을 분산환경에서도 실행할 수 있도록 구현해놓은 라이브러리

- 여러개의 Decision Tree를 조합해서 사용하는 앙상블 알고리즘

- 구조화 되어있거나 표로 정리된 데이터셋에서 분류/회귀 에측모형을 만드는데 가장 많이 쓰임

- 다른 Gradient Boosting 모델들에 비해 속도 매우 빠름

- 모델 performance가 매우 좋음 (대부분의 캐글 상위 입상자들이 XGBoost 모델 사용) 

#### XGBoost 모델의 파라미터
- max_depth: 트리 모델의 최대 깊이. 값이 높을수록 더 복잡한 트리 모델 생성, 과적합의 원인이 될 수 있음.

- eta: 딥러닝에서의 learning rate와 같은 개념. 0과 1 사이의 값을 가지며, 값이 너무 높으면 학습이 잘 되지 않을 수 있고 값이 너무 낮으면 학습이 느릴 수 있음.

- colsample_bytree: 트리를 생성할 때 훈련 데이터에서 변수를 샘플링해주는 비율. 모든 트리는 전체 변수의 일부만을 학습하여 서로의 약점을 보완. 보통 0.6~0.9 사용.

- colsample_bylevel: 트리의 레벨 별로 훈련 데이터의 변수를 샘플링해주는 비율. 보통 0.6~0.9 사용.


*참고:
모델의 파라미터 튜닝 작업에 너무 많은 시간을 투자하기 보다는 적당한 수준의 파라미터 튜닝 + 많은 시간을 피처 엔지니어링에 투자하여 양질의 변수를 얻자!

In [ ]:
# XGBoost 모델 parameter를 설정한다.
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

In [ ]:
# 훈련, 검증 데이터를 XGBoost 형태로 변환한다.
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)


In [ ]:
# XGBoost 모델을 훈련 데이터로 학습한다!
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)


In [ ]:
# 학습한 모델을 저장한다.
import pickle 
#pickle 모듈: 텍스트 상태의 데이터가 아닌 파이썬 객체 자체를 파일로 저장
#pickle.dump(객체, 파일) 로 저장하고
#pickle.load(파일) 로 로딩

pickle.dump(model, open("../model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit

### 교차검증
이번 경진대회의 평가척도인 MAP@7을 사용하여 성능 수준을 확인.
(교차 검증 과정에서 경진대회에서 실제로 사용되는 평가 척도를 사용하는 것은 매우 중요)

검증 데이터에서 얻을 수 있는 최고점 0.042663 < 1인 이유: 검증 데이터의 모든 고객이 신규 구매를 하지는 않았기 때문.

예를 들어, 100명의 고객 중 10명만이 신규 구매 => 10명을 정확히 예측해도 10%의  MAP@7 점수를 받게 됨.

따라서, 검증 데이터의 MAP@7 최고 점수를 감안하여 학습 모델의 성능을 평가해야함.

In [ ]:
# MAP@7 평가 척도를 위한 준비작업이다.
# 고객 식별 번호를 추출한다.
vld = trn[trn['fecha_dato'] == vld_date] #훈련 데이터에서 검증 데이터 분리
ncodpers_vld = vld.as_matrix(columns=['ncodpers']) #검증 데이터에서 고객식별번호 ncodpers 

# 검증 데이터에서 신규 구매를 구한다.
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]   #신규 구매 여부 (이 값이 1이면 신규 구매한 것)
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]
# 고객별 신규 구매 정답 값을 add_vld_list에 저장하고, 총 count를 count_vld에 저장한다.
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

In [ ]:
# 검증 데이터에서 얻을 수 있는 MAP@7 최고점을 미리 구한다. (0.042663)
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

In [ ]:
# 검증 데이터에 대한 예측 값을 구한다.
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit) #예측값

In [ ]:
# 저번 달에 보유한 제품은 신규 구매가 불가하기 때문에, 확률값에서 미리 1을 빼준다
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])


In [ ]:
# 검증 데이터 예측 상위 7개를 추출한다.
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])


In [ ]:
# 검증 데이터에서의 MAP@7 점수를 구한다. (0.036466)
print(mapk(add_vld_list, result_vld, 7, 0.0))


검증 데이터 최고 점수가 0.042663임을 감안한다면, Baseline 모델의 정확도는 0.036466/0.042663=0.85로 약 85% 수준.

앞서 교차 검증에서 훈련 데이터의 일부를 도려내어 검증 데이터로 사용하였음.

이번에는 테스트 데이터에 대해서 조금이라도 좋은 성능을 내기 위하여 훈련 데이터와 검증 데이터를 합친 전체 데이터에 대하여 모델 다시 학습.

파라미터는 교차 검증을 통해 찾아낸 최적의 파라미터를 사용하되 트리의 개수를 늘어난 검증 데이터만큼 증가

In [ ]:
# XGBoost 모델을 전체 훈련 데이터로 재학습한다!
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]

In [ ]:
# 트리 개수는 늘어난 데이터 양만큼 비례해서 증가한다.
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))


In [ ]:
# XGBoost 모델 재학습!
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)


In [ ]:
# 변수 중요도를 출력해본다. 예상하던 변수가 상위로 올라와 있는가?
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)


In [ ]:
# 캐글 제출을 위하여 테스트 데이터에 대한 예측 값을 구한다.
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])


In [ ]:
# 제출 파일을 생성한다.
submit_file = open('../model/xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers,added_products\n')
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))
